# Optimizing Model Parameters
Now that we have a model and data, it's time to train, validate and test the model by optimizing its parameters to the data. Training a model is an iterative process; in each iteration (called an *epoch*) the model makes a guess about the output, calculates the error in its guess (*loss*), collects the derivatives for the error with respect to its parameters ( as we saw in the [previous section](https://pytorch.org/tutorials/beginner/basics/autograd_tutorial.html)), and **optimizes** these parameters using gradient descent. For a more detailed walkthrough of this process, check out the video on [backpropagation from 3Blue1Brown](https://www.youtube.com/watch?v=tIeHLnjs5U8).

## Prerequisite Code
We load the code from the previous secitons on [Datasets & DataLoaders](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html) and [Build Model](https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html).

In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

train_data = datasets.FashionMNIST(
    root = '../data',
    train = True,
    download = True,
    transform = ToTensor(),
)

test_data = datasets.FashionMNIST(
    root = '../data',
    train = False,
    download = True,
    transform = ToTensor(),
)

train_loader = DataLoader(train_data, batch_size=64)
test_loader = DataLoader(test_data, batch_size=64)


softargmax = nn.Softmax
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        
        return logits
    
model = NeuralNetwork()

## Hyperparameters
Hyperparameters are adjustable parameters that let you control the model optimization process. Different hyperparameter values can impact model training and convergence rates (read more about [hyperparameter tuning](https://pytorch.org/tutorials/beginner/hyperparameter_tuning_tutorial.html)).

We define the following hyperparameters for training:
- **Number of Epochs**: The number of times to iterate over a dataset
- **Batch Size**: The number of data samples propageted through the network before the parameters are updated
- **Learning Rate**: How much to update the model parameters at each batch/epoch. Smaller values yield slower learning speeds, while larger values carry risks of overshooting the optima resulting in unpredictable behaviours.

In [2]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

## Optimization Loops
Once we set our hyperparameters, we can train and optimize our model with an optimization loop. Each iteration of the optimization loop is called an **epoch**.

Each epoch consists of two main parts:
- **The train loop**: iterate over the train data and try to converge parameters to the optimum
- **The validation/test loop**: iterate over the test data to check if the model is improving

Let's briefly familiarize ourselves with some of the concepts used in the training loop.

### Loss function
When presented with some training data, our untrained network is likely to not give correct answers. **Loss function** measures the degree of dissimilarity of obtained results to the target values, and it is the loss function that we want to minimize during training. To calculate the loss, we make a prediction using the inputs of our given data sample and compare it against the true data label value.

Common loss functions include [nn.MSELoss](https://pytorch.org/docs/stable/generated/torch.nn.MSELoss.html#torch.nn.MSELoss) (Mean Square Error) for regression tasks, and [nn.NLLLoss](https://pytorch.org/docs/stable/generated/torch.nn.NLLLoss.html#torch.nn.NLLLoss) (Negative Log Likelihood) for classification tasks. [nn.CrossEntropyLoss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html#torch.nn.CrossEntropyLoss) is a combination of `nn.LogSoftmax` and `nn.NLLLoss` for multi-class classification.

We pass our model's output logits to `nn.CrossEntropyLoss`, which will normalize the logits and compute a prediction error.

In [3]:
# Initialize the loss function
loss = nn.CrossEntropyLoss()

### Optimizer
Optimization is the process of adjusting model parameters to reduce model error in each training step. **Optimization algorithms** define how this process is performed (in this example we use Stochastic Gradient Descent SGD). All optimization logic is encapsulated in the `optimizer` object. Here, we use the SGD optimizer; additionally, there are many [different optimizers](https://pytorch.org/docs/stable/optim.html) available in PyTorch such as ADAM and RMSProp, that work better for different kinds of models and data.

We initialize the optimizer by registering the model's parameters that need to be trained, and passing the learning rate hyperparameter.

In [4]:
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

Inside the training loop, optimization happens in three steps:
- Call `optimizer.zero_grad()` to reset the gradients of the model parameters. Gradients by default add up; to prevent double-counting, we explicitly zero them at each iteration.
- Backpropagate the prediction loss with a call to `loss.backward()`. PyTorch deposits the gradients of the loss w.r.t. each parameter.
- Once we have our gradients, we call `optimizer.step()` to adjust the parameters by the gradients collected in the backward pass.

## Full Implementation
We define `train_loop` that loops over our optimization code, and `test_loop` that evaluates the model's performance against our test data.

In [5]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch , (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # Back propagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # For every 100th batch
        if batch % 100 == 0:
            loss, current = loss.item(), batch*len(X)
            print (f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            
    test_loss /= num_batches
    correct /= size
    print (f"Test Error: \n Accuracy: {100*correct:>0.1f}%, Avg loss: {test_loss:>8f} \n")

We initialize the loss function and optimizer, and pass it to `train_loop` and `test_loop`. Feel free to increase the number of epochs to track the model's improving performance.

In [6]:
epochs = 10
for epoch in range(epochs):
    print (f"Epoch {epoch+1}\n----------------------------------------")
    train_loop(train_loader, model, loss, optimizer)
    test_loop(test_loader, model, loss)
print ("Done!")

Epoch 1
----------------------------------------
loss: 2.312463 [    0/60000]
loss: 2.297698 [ 6400/60000]
loss: 2.280557 [12800/60000]
loss: 2.269430 [19200/60000]
loss: 2.247051 [25600/60000]
loss: 2.226167 [32000/60000]
loss: 2.226198 [38400/60000]
loss: 2.192791 [44800/60000]
loss: 2.195713 [51200/60000]
loss: 2.165344 [57600/60000]
Test Error: 
 Accuracy: 46.3%, Avg loss: 2.157935 

Epoch 2
----------------------------------------
loss: 2.168652 [    0/60000]
loss: 2.162374 [ 6400/60000]
loss: 2.108412 [12800/60000]
loss: 2.125664 [19200/60000]
loss: 2.074285 [25600/60000]
loss: 2.010631 [32000/60000]
loss: 2.043245 [38400/60000]
loss: 1.958803 [44800/60000]
loss: 1.968870 [51200/60000]
loss: 1.899127 [57600/60000]
Test Error: 
 Accuracy: 60.1%, Avg loss: 1.898049 

Epoch 3
----------------------------------------
loss: 1.928307 [    0/60000]
loss: 1.899060 [ 6400/60000]
loss: 1.788612 [12800/60000]
loss: 1.833482 [19200/60000]
loss: 1.719269 [25600/60000]
loss: 1.665149 [32000/60